#clone darknet repo

In [1]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15339, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 15339 (delta 1), reused 6 (delta 1), pack-reused 15331
Receiving objects: 100% (15339/15339), 13.93 MiB | 16.78 MiB/s, done.
Resolving deltas: 100% (10381/10381), done.


Also,we should change makefile to have GPU and OPENCV enabled and verify CUDA

In [2]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


# make darknet

In [3]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

Get yolov3 pretrained coco dataset weights

In [4]:
# get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2021-10-31 09:56:07--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  22.8MB/s    in 11s     

2021-10-31 09:56:18 (21.4 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [5]:
def showimg(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  img = cv2.imread(path)
  h, w = img.shape[:2]
  resized_img = cv2.resize(img,(3*w, 3*h), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
  plt.show()

Connect Google Drive

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Mounted at /content/gdrive


In [ ]:
!cp /content/gdrive/MyDrive/3710/train.zip /content/darknet/data/

In [ ]:
!unzip /content/darknet/data/train.zip -d /content/darknet/data/

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/darknet/data/obj/ISIC_0000103.jpg  
  inflating: /content/darknet/data/obj/ISIC_0000103.txt  
  inflating: /content/darknet/data/obj/ISIC_0000104.jpg  
  inflating: /content/darknet/data/obj/ISIC_0000104.txt  
  inflating: /content/darknet/data/obj/ISIC_0000105.jpg  
  inflating: /content/darknet/data/obj/ISIC_0000105.txt  
  inflating: /content/darknet/data/obj/ISIC_0000107.jpg  
  inflating: /content/darknet/data/obj/ISIC_0000107.txt  
  inflating: /content/darknet/data/obj/ISIC_0000108.jpg  
  inflating: /content/darknet/data/obj/ISIC_0000108.txt  
  inflating: /content/darknet/data/obj/ISIC_0000109.jpg  
  inflating: /content/darknet/data/obj/ISIC_0000109.txt  
  inflating: /content/darknet/data/obj/ISIC_0000110.jpg  
  inflating: /content/darknet/data/obj/ISIC_0000110.txt  
  inflating: /content/darknet/data/obj/ISIC_0000111.jpg  
  inflating: /content/darknet/data/obj/ISIC_0000111.txt  
  inflating: /content/darknet/data/obj/ISIC_0

Get the image path from the train folder

In [ ]:
import os
def train_path():
  image_files = []
  os.chdir(os.path.join("/content/darknet/data", "train"))
  for filename in os.listdir(os.getcwd()):
      if filename.endswith(".jpg"):
          image_files.append("/content/darknet/data/train/" + filename)
  os.chdir("..")
  with open("train.txt", "w") as outfile:
      for image in image_files:
          outfile.write(image)
          outfile.write("\n")
      outfile.close()
  os.chdir("..")

train_path()

In [ ]:
!cp /content/gdrive/MyDrive/3710/obj.name /content/darknet/data
!cp /content/gdrive/MyDrive/3710/obj.data /content/darknet/data
!cp /content/gdrive/MyDrive/3710/yolov3_custom_3710.cfg /content/darknet/cfg

In [6]:
!wget http://pjreddie.com/media/files/darknet53.conv.74

URL transformed to HTTPS due to an HSTS policy
--2021-10-31 09:59:25--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  23.4MB/s    in 7.3s    

2021-10-31 09:59:33 (21.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# Start training

In [ ]:
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov3_custom_3710.cfg darknet53.conv.74 -dont_show

流式输出内容被截断，只能显示最后 5000 行内容。
 total_bbox = 247837, rewritten_bbox = 0.000000 % 

 3926: 0.088586, 0.103208 avg loss, 0.000010 rate, 5.470399 seconds, 251264 images, 0.172530 hours left
Loaded: 0.000051 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.799922), count: 3, class_loss = 0.322512, iou_loss = 0.140843, total_loss = 0.463355 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.796558), count: 1, class_loss = 0.139221, iou_loss = 0.030072, total_loss = 0.169293 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000012, iou_loss = 0.000000, total_loss = 0.000012 
 total_bbox = 247841, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.863406), count: 4, class_loss = 0.810068, iou_loss = 0.081452, total_loss = 0.891520 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 9